In [ ]:
#Import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

**PSD, FFT**

In [ ]:
#Name of the file############################################################
name='best_777_T_Bz=5A_Bx=0.08A_exc810_det852.3_5mW_5K_sigma+_PEM_(30min).txt'
#############################################################################

exp_data = np.loadtxt(name, skiprows=1, delimiter='\t')

t=exp_data[:,0]
rho=exp_data[:,3]

#FFT
sp = np.fft.fft(rho)
freq_fft = np.fft.fftfreq(t.shape[-1])
power_fft=(sp.real)**2
power_fft[0]=0

freq, power = signal.periodogram(rho, scaling='spectrum')
#freq, power = signal.welch(rho, scaling='spectrum')

font = {'size': 20}

#Main signal
plt.figure(figsize=(15, 5))
plt.title(r'Signal')
plt.ylabel("$p_c$(%)")
plt.xlabel("Time (s)")
plt.plot(t, rho, 'g', lw=2)
plt.xlim([0,max(t)])
plt.rc('font', **font)

#Power spectrum
plt.figure(figsize=(15, 5))
plt.title(r'Power spectral density')
plt.semilogy(freq, power, 'k')
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
axes = plt.gca()
axes.set_xlim([0,0.5])
axes.set_ylim([10**-15,max(power)+max(power)*10])

#FFT
plt.figure(figsize=(15, 5))
plt.title(r'FFT')
plt.plot(freq, power, 'r')
axes = plt.gca()
axes.set_xlim([0,0.5])
axes.set_ylim([0,max(power)+max(power)*0.1])
axes.fill_between(freq, 0, power, facecolor='red')
plt.xlabel(r'frequency [Hz]')
plt.ylabel(r'(arb. un.)')
plt.yticks([])

**Autocorrelation**

The autocorrelation of a signal describes the similarity of a signal against a time-shifted version of itself. For a signal $x$, the autocorrelation $r$ is:
$$r(k)=∑_nx(n)x(n−k)$$
 
In this equation, $k$ is often called the lag parameter.  $r(k)$ is maximized at  $k=0$ and is symmetric about $k$.
The autocorrelation is useful for finding repeated patterns in a signal. For example, at short lags, the autocorrelation can tell us something about the signal's fundamental frequency. For longer lags, the autocorrelation may tell us something about the tempo of a musical signal.

In [ ]:
N = len(rho)
fvi = np.fft.fft(rho, n=2*N)
acf = np.real(np.fft.ifft(fvi * np.conjugate(fvi))[:N])
acf = acf/(N - np.arange(N))

plt.figure(figsize=(15, 5))
plt.title(r'Autocorrelation')
plt.xlabel(r'Time delay $t$')
plt.ylabel(r'$r(t)$')
plt.plot(t[0:len(acf)-int(len(acf)/2)], acf[0:len(acf)-int(len(acf)/2)], 'r')
plt.grid(color='black', linewidth=1)
plt.rc('font', **font)


**Embedding time**

The time delayed mutual information was suggested by Fraser and Swinney as a tool to determine a reasonable delay: Unlike the autocorrelation function, the mutual information takes into account also nonlinear correlations. One has to compute
$$
S = - \sum_{ij} p_{ij}(\tau) ln \frac{p_{ij}(\tau)}{p_ip_j}, 
$$
where for some partition on the real numbers $p_{ij}$ is the probability to find a time series value in the $i$-th interval, and $p_{ij}(\tau)$ is the joint probability that an observation falls into the $i$-th interval and the observation time $\tau$ later falls into the $j$-th. In theory this expression has no systematic dependence on the size of the partition elements and can be quite easily computed. There exist good arguments that if the time delayed mutual information exhibits a marked minimum at a certain value of $\tau$, then this is a good candidate for a reasonable time delay. However, these arguments have to be modified when the embedding dimension exceeds two. Moreover, as will become transparent in the following sections, not all applications work optimally with the same delay. Since we are not really interested in absolute values of the mutual information here but rather in the first minimum, the minimal implementation given here seems to be sufficient.